In [1]:
__nbid__ = '0049'
__author__  = 'Mike Fitzpatrick <mike.fitzpatrick@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>, David Herrera <david.herrera@noirlab.edu>, Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20251205' # aaaammdd
__keywords__ = ['query', 'vospace', 'mydb']

# Cómo utilizar el servicio *Query Client* de Data Lab

#### (_Traducción de [How_to_use_the_Data_Lab_QueryClient.ipynb](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/QueryClient/How_to_use_the_Data_Lab_QueryClient.ipynb)_)

### Tabla de contenido
* [Resumen](#summary)
* [Avisos Legales y Atribuciones](#attribution)
* [Importaciones y configuración](#imports)
* [Autenticación](#auth)
* [Revisión: Plantilla general para una consulta sencilla en SQL](#review)
* [Consulta de ejemplo rápido](#query)
* [Guardar en almacenamiento virtual VOSpace](#savevospace)
* [Guardar en la base de datos remota MyDB](#savemydb)
* [Importar datos a una tabla MyDB con QueryClient](#importmydb)
* [Un ejemplo de consulta asincrónica](#async)
* [Usando perfiles](#profiles)

<a class="anchor" id="summary"></a>

# Resumen

Este notebook documenta cómo consultar el laboratorio de datos a través del servicio de consulta del cliente. Para obtener la documentación completa, consulte la <a href="https://datalab.noirlab.edu/docs/api/dl.html#module-dl.queryClient">documentación de API</a>.


### La clase *queryClient*


Todas las consultas se ejecutan a través del método *queryClient.query()* de la clase *queryClient*. Esto toma como argumentos:

| Argumento | Descripción | Opcional | Valor predeterminado | Valores admitidos |
|----------|-------------|----------|----------------|----------------|
| adql | La consulta que se enviará al servicio TAP | No | None | cadena |
| SQL | La consulta que se enviará directamente a la base de datos | Sí | None | cadena |
| fmt | El formato solicitado (si corresponde) | Sí | ascii | ascii,csv,fits,hdf5,votable |
| out | La ubicación guardada (si corresponde) | Sí | None | nombre de archivo local, *vos://nombre_de_archivo*, *mydb://nombre_de_tabla* |
| async_ | Indica si la consulta es asíncrona | Sí | False | True/False |


Todos los argumentos son opcionales, excepto que se debe proporcionar uno de *adql* o *sql*.  La distinción entre estos dos parámetros está en cómo *QueryClient* ejecuta la consulta: si se proporciona *adql*, la consulta se envía al servicio TAP (Protocolo de acceso a tablas), si se proporciona *sql*, la consulta se envía directamente a la base de datos. La elección de la ejecución depende de si la cadena de consulta contiene funciones específicas de ADQL o construcciones SQL o extensiones de base de datos que el servicio TAP no comprende.  Para consultas grandes también puede haber una diferencia de rendimiento dependiendo de dónde y cómo se guardan los resultados.

#### Formatos de salida
Los resultados se pueden devolver como espacios en blanco delimitados (*ascii*), CSV (*csv*), objeto FITS (*fits*), HDF5 (*hdf5*) o en formato VOTable (*votable*). Tenga en cuenta que si los resultados se guardan en la base de datos del usuario, se ignora el formato de salida.

#### Guardando resultados
Si no se especifica ninguna ubicación para guardar (sin parámetro *out*), los resultados se devuelven directamente. Una ubicación para guardar que comienza con el identificador 'vos://' indica una ubicación en el almacenamiento virtual del usuario para guardar el resultado. Una ubicación para guardar que comienza con el identificador 'mydb://' indica que los resultados se guardarán en una tabla en la base de datos remota del usuario (MyDB).

#### Consultas asincrónicas
Las consultas largas deben ejecutarse de forma asincrónica y el servicio puede rechazar cualquier intento de consulta sincrónica si el tiempo de consulta proyectado es demasiado largo. Se puede enviar una consulta de forma asincrónica estableciendo el parámetro *async_* en *True*. Luego se devolverá una identificación del trabajo.


El estado de una consulta asincrónica se puede comprobar con *queryClient.status(jobid)*. Un valor de retorno de 'COMPLETADO' indica que la consulta ha finalizado. Un valor de retorno de 'ERROR' indica que hubo un problema con la consulta.

Los resultados de una consulta asincrónica (suponiendo que no se guardaron ni en el almacenamiento virtual del usuario ni en la base de datos remota) se pueden recuperar una vez que la consulta se haya completado con *result = qc.query(adql=query,async_=True,wait=True,poll=1,verbose=1)*

### Del código Python

El servicio de cliente de consultas se puede llamar desde código Python usando el módulo *datalab*. Esto proporciona métodos para acceder a las diversas funciones del cliente de consultas en el subpaquete *QueryClient*. Consulte la información [aquí](https://github.com/astro-datalab/datalab/blob/master/README.md).

Las consultas también se pueden ejecutar desde la línea de comando, por ejemplo, en su máquina local, utilizando la utilidad de línea de comandos de datalab. Lea sobre esto en nuestro repositorio de GitHub [aquí](https://github.com/astro-datalab/datalab).

<a class="anchor" id="attribution"></a>
# Avisos Legales y atribuciones

Avisos Legales
-----------
Tome en cuenta que usar el Astro Data Lab constituye un acuerdo con nuestros [Avisos Legales](https://datalab.noirlab.edu/disclaimers.php) mínimos.

Reconocimientos
---------------
Si ud. usa el **Astro Data Lab** en sus publicaciones de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios de datos proveeidos por el Astro Data Lab, el cual es parte del Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) del NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc."(Asociación de Universidaddes para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

Si utiliza **SPARCL junto con la plataforma de Astro Data Lab** (por medio de JupyterLab, línea de comando o interfaz de la web) en su publicación de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios o datos proporcionados por el "SPectra Analysis and Retrievable Catalog Lab" (SPARCL) (Laboratorio de Análisis y Catálogo Recuperable de Espectros) y el Astro Data Lab, ambos pertenecientes al Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) de NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc." (Asociación de Universidades para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

En cualquiera de los casos, **por favor cite las siguientes publicaciones**:

* Publicación del concepto de Data Lab: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411.

Si hace referencia al Jupyterlab / Jupyter notebooks de Data Lab, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097.

Si publica en una revista de la AAS, agregue también la palabra clave `\facility{Astro Data Lab}`

Y si está usando SPARCL, por vor agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576.

La biblioteca de NOIRLab mantiene [listas de reconocimientos apropiados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar cuando se hacen publicaciones utilizando los recursos, servicios o datos del Laboratorio.

---- **Versión en Inglés** ----


# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.


<a class="anchor" id="imports"></a>
# Importaciones y configuración

Esta es la configuración necesaria para utilizar el cliente de consultas. Lo primero que debe hacer es importar los módulos de Python relevantes.

In [2]:
# Biblioteca estándar
from getpass import getpass

# De Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert

# Autenticación
Se puede acceder a gran parte de la funcionalidad de Data Lab sin iniciar sesión explícitamente (el servicio utiliza un inicio de sesión anónimo). Pero algunas capacidades, por ejemplo guardar los resultados de sus consultas en su espacio de almacenamiento virtual, requieren un inicio de sesión (es decir, necesitará una cuenta de usuario registrada).

Si necesita iniciar sesión en Data Lab, descomente la celda a continuación y ejecútela:

In [ ]:
# token = ac.login(input("Ingrese el nombre de usuario: (+ENTER) "), getpass("Ingrese la contraseña: (+ENTER) "))
#ac.whoAmI()

<a class="anchor" id="review"></a>
# Reseña: Plantilla general para una consulta sencilla en SQL

### SQL es una forma de describir a una base de datos lo que quieres de ella.
Plantilla general para una consulta sencilla escrita en SQL

``` SQL
SELECT something
FROM database.table
WHERE constraints
LIMIT 100
```

### Consulte nuestro notebook de introducción [JupyterPythonSQL101](https://github.com/astro-datalab/notebooks-latest-es/blob/master/01_ComenzandoConDataLab/01_JupyterPythonSQL101.ipynb) para obtener más información sobre esto.

<a class="anchor" id="query"></a>
# Una consulta rápida

Digamos que queremos obtener las magnitudes g,r,i de 10 objetos en el conjunto de datos SMASH DR1 y recuperar los resultados como una cadena con formato CSV:

In [3]:
query = 'SELECT gmag, rmag, imag FROM smash_dr1.object WHERE gmag<99 AND rmag<99 AND imag<99 LIMIT 10'
response = qc.query(sql = query, fmt = 'csv')
print (response)

gmag,rmag,imag
11.915434,11.797816,11.789193
11.937625,11.509244,11.325349
12.069924,11.693381,11.55774
12.074745,16.537102,17.018686
12.124366,11.802776,11.616948
12.135513,11.762072,11.546856
12.184068,11.844554,11.719124
12.210214,12.188986,12.318025
12.227514,11.763244,11.506152
12.249985,11.839742,11.703437



<a class="anchor" id="savevospace"></a>
# Guardar resultados en el almacenamiento virtual VOSpace

VOSpace es un espacio de almacenamiento conveniente para que los usuarios guarden su trabajo. Puede almacenar cualquier dato o tipo de archivo.

Ahora queremos guardar los resultados de la misma consulta en nuestro espacio de almacenamiento virtual.  Al colocar la consulta en un bloque de prueba, podemos detectar errores al ejecutar la consulta.  Tenga en cuenta que ejecutar esta celda varias veces generará un error y usaremos el Cliente de Storage Manager para eliminar el archivo una vez que hayamos terminado.

In [4]:
try:
    response = qc.query (sql=query, fmt='csv', out='vos://examplemags.csv')
except Exception as e:
    # Maneje cualquier error en la consulta.  Al ejecutar esta celda varias veces con el mismo
    # archivo de salida, o mediante el uso de una declaración SQL falsa, puede ver varios mensajes de error.
    print (e.message)
else:
    if response is not None: 
        print (response)        
    else:
        print ("OK")

OK


Asegurémonos de que el archivo se haya creado en VOSpace:

In [5]:
sc.ls(name='vos://examplemags.csv')

'examplemags.csv'

Ahora eliminemos el archivo que acabamos de crear:

In [6]:
sc.rm (name='vos://examplemags.csv')

'OK'

Asegurémonos de que el archivo se haya eliminado de VOSpace:

In [7]:
sc.rm (name='vos://examplemags.csv')

'A Node does not exist with the requested URI.'

<a class="anchor" id="savemydb"></a>
# Guardar resultados en la base de datos remota MyDB
MyDB es una útil base de datos relacional remota por usuario del sistema operativo que puede almacenar tablas de datos. Más aun, los resultados de las consultas se pueden guardar directamente en MyDB, como mostramos en el siguiente ejemplo:

In [8]:
try:
    response = qc.query (sql=query, fmt='csv', out='mydb://examplemags')
except Exception as e:
    # Maneje cualquier error en la consulta.  Al ejecutar esta celda varias veces con el mismo
    # archivo de salida, o mediante el uso de una declaración SQL falsa, puede ver varios mensajes de error.
    print (e.message)
else:
    if response is not None: 
        print (response)         
    else:
        print ("OK")

OK


Asegúrese de que la tabla se haya guardado en MyDB llamando a la función *mydb_list()*:

In [9]:
print(qc.mydb_list(),"\n")

allwisetab
basic_result
desi_dr1_survey_progress_q1
examplemags
gals
largetable
lstab
table1
tbl_stat
test1
testresult2
usno_objects
y6gold_test
 



Ahora eliminemos la tabla de nuestra MyDB.

In [10]:
qc.mydb_drop('examplemags')

'OK'

Asegúrese de que se haya eliminado llamando nuevamente a la función *mydb_list()*:

In [11]:
print(qc.mydb_list(),"\n")

allwisetab
basic_result
desi_dr1_survey_progress_q1
gals
largetable
lstab
table1
tbl_stat
test1
testresult2
usno_objects
y6gold_test
 



<a class="anchor" id="importmydb"></a>
# Importar datos a una tabla MyDB con QueryClient

Los usuarios pueden usar la función *mydb_import* para importar datos guardados en una computadora local o importar datos desde VOSpace a una tabla de datos MyDB. Los datos deben tener el formato de un archivo CSV o de un objeto Dataframe de Pandas para poder cargarlos en MyDB.

Primero, consultemos algunos datos de la base de datos de Data Lab y guardémoslos localmente como un archivo CSV:

In [12]:
query = "select * from gaia_dr1.gaia_source limit 10"
qc.query (sql=query, fmt='csv', out='./gaiaresult.csv')

'OK'

A continuación usaremos la función *mydb_import* para importar el archivo CSV almacenado localmente en una tabla MyDB:

In [13]:
qc.mydb_import('testresult','./gaiaresult.csv')

'OK'

Asegurémonos de que esté ahí llamando a la función *mydb_list()*:

In [14]:
print(qc.mydb_list(),"\n")

allwisetab
basic_result
desi_dr1_survey_progress_q1
gals
largetable
lstab
table1
tbl_stat
test1
testresult
testresult2
usno_objects
y6gold_test
 



De manera similar, podemos usar la función *mydb_import* para importar datos de VOSpace a una tabla MyDB:

In [15]:
qc.mydb_import('testresult2','vos://newmags.csv')

'OK'

In [16]:
print(qc.mydb_list(),"\n")

allwisetab
basic_result
desi_dr1_survey_progress_q1
gals
largetable
lstab
table1
tbl_stat
test1
testresult
testresult2
usno_objects
y6gold_test
 



Finalmente, para fines de limpieza, eliminemos las dos tablas que acabamos de importar a MyDB usando la función *mydb_drop*:

In [17]:
qc.mydb_drop('testresult')
qc.mydb_drop('testresult2')

'relation "testresult2" not known'

Y asegurémonos de que las dos tablas se eliminaron de MyDB usando la función *mydb_list()*:

In [18]:
print(qc.mydb_list(),"\n")

allwisetab
basic_result
desi_dr1_survey_progress_q1
gals
largetable
lstab
table1
tbl_stat
test1
testresult2
usno_objects
y6gold_test
 



<a class="anchor" id="async"></a>
# Una consulta asincrónica

Ahora queremos ejecutar una consulta más larga, por ejemplo, contar el número total de objetos en USNO-B1, y debemos hacerlo de forma asincrónica. Para hacer esto, enviaremos la consulta normalmente pero con el argumento *async_* indicado.

In [19]:
query = 'SELECT count(*) FROM usno.b1'

In [20]:
result = qc.query(sql=query,async_=True,wait=True,poll=15,verbose=1)

EXECUTING
Status = EXECUTING; elapsed time: 15, timeout in 285
EXECUTING
Status = EXECUTING; elapsed time: 30, timeout in 270
EXECUTING
Status = EXECUTING; elapsed time: 45, timeout in 255
EXECUTING
Status = EXECUTING; elapsed time: 60, timeout in 240
EXECUTING
Status = COMPLETED; elapsed time: 75, timeout in 225
Retrieving results

Total time: 0.03s for 17.00 B


In [21]:
print(result)

COUNT
1045175762



### Consulte nuestro notebook [JupyterPythonSQL101](https://github.com/astro-datalab/notebooks-latest-es/blob/master/01_ComenzandoConDataLab/01_JupyterPythonSQL101.ipynb) para obtener más información sobre cómo obtener un jobid, verificar el estado hasta 'COMPLETED' y recuperar resultados con qc.resultados().

<a class="anchor" id="profiles"></a>
# Usando perfiles

A veces, diferentes conjuntos de datos (o versiones del mismo conjunto de datos) pueden residir en diferentes servidores backend y es posible que un usuario desee trabajar explícitamente con un conjunto de datos (normalmente más antiguo). En algunos casos estos servidores serán utilizados únicamente por desarrolladores o aquellos con acceso restringido. Además, se puede acceder a los servicios TAP externos desde Data Lab. En ambos casos de uso, la función *qc.profiles()* resulta útil.

Lo primero que debemos hacer es ver qué perfiles están disponibles:

In [22]:
profilelist = qc.list_profiles()
print (profilelist)

       GALEX-DR6   GALEX DR6 TAP service (29 Tables, TAP Only)
            GAVO   GAVO Data Center TAP service (149 Tables, TAP Only)
         HEASARC   HEASARC Xamin TAP Service (921 Tables, TAP Only)
            IRSA   IRSA TAP Service (478 Tables, TAP Only)
        SDSS-DR9   SDSS DR9 TAP service (92 Tables, TAP Only)
          SIMBAD   SIMBAD TAP service (47 Tables, TAP Only)
    STScI-RegTAP   STScI Registry TAP service (18 Tables, TAP Only)
          Vizier   TAP Vizier query engine (34381 Tables, TAP Only)



Lo que hay que tener en cuenta en el resultado aquí son nombres como '*GAVO*', '*SDSS-DR9*', etc;  **Estos perfiles se refieren a servicios TAP externos a los que se puede acceder mediante la interfaz de Query Manager.** Solo unos pocos están configurados en este momento mientras trabajamos en formas de descubrir automáticamente más de 100 servicios de este tipo y proporcionar listas útiles de lo que contienen.

Veamos cómo podemos consultar uno y guardar el resultado en nuestro VOSpace de Almacenamiento Virtual:

In [23]:
qc.set_profile('GAVO')
query = 'SELECT top 10 * FROM sdssdr7.sources'
response = qc.query(adql = query, fmt = 'csv', out="vos://gavo_out.csv")
print (response)

OK


En este caso consultamos una tabla SDSS DR7 en el servicio TAP de GAVO en Heidelberg.

Asegurémonos de que se haya guardado en VOSpace:

In [24]:
listing = sc.ls (name='vos://',format='long')
print (listing)

drwxrwxr-x  demo01       0  27 Apr 2024 18:32  HIPASS_images_most_massive/
drwxrwxr-x  demo01       0  27 Apr 2024 18:32  HIPASS_spectra_most_massive/
-rw-rw-r-x  demo01  1836731  16 Dec 2021 14:45  LS_DR8N_part2.csv
-rw-rw-r-x  demo01  2954880  16 Dec 2021 14:45  cutout.fits
-rw-rw-r-x  demo01  7936244  01 Jan 2026 23:51  gaia100pc.csv
-rw-rw-r-x  demo01    4002  13 Oct 2025 18:01  gavo_out.csv
-rw-rw-r-x  demo01    4850  02 Jan 2026 21:34  mags.csv
-rw-rw-r-x  demo01  376550  13 Oct 2025 17:21  ps1.vot
drwxrwxr-x  demo01       0  16 Dec 2021 14:45  public/
-rw-rw-r-x  demo01     190  13 Oct 2025 18:01  redrock-main-dark-26067.fits
drwxrwxr-x  demo01       0  16 Dec 2021 14:45  results/
-rw-rw-r-x  demo01  17011851  25 Feb 2025 13:58  test10k.csv
-rw-rw-r-x  demo01  66906667  25 Feb 2025 13:27  test_large.csv
-rw-rw-r-x  demo01   33001  13 Oct 2025 17:15  timeseries.csv
drwxrwxr-x  demo01       0  16 Dec 2021 14:45  tmp/



Podemos cargar el conjunto de datos y p.e. convertir a un marco de datos de Pandas, como este:

In [25]:
data = sc.get(fr = 'vos://gavo_out.csv', to = '')
df = convert(data)
df

,objID,run,rerun,camcol,fieldId,obj,ra,dec,raErr,decErr,...,offsetRa_i,offsetDec_i,i,err_i,epoch_i,offsetRa_z,offsetDec_z,z,err_z,epoch_z
0,758883089249337439,6182,648,1,758883089249337344,95,331.786804,41.387806,0.000117,0.000113,...,-0.000064,0.000065,19.978480,0.092936,2006.409,4.666667e-07,-2.388889e-07,22.883123,2.086310,2006.409
1,758883089249337433,6182,648,1,758883089249337344,89,331.780687,41.389283,0.000035,0.000030,...,-0.000111,-0.000117,20.797735,0.097531,2006.409,-1.363639e-04,3.201944e-05,20.136680,0.202043,2006.409
2,758883089249337434,6182,648,1,758883089249337344,90,331.784089,41.389143,0.000033,0.000039,...,-0.000402,0.000139,21.472952,0.153602,2006.409,-3.638889e-07,3.530556e-06,20.962902,0.351456,2006.409
3,758883089249337438,6182,648,1,758883089249337344,94,331.793057,41.390098,0.000036,0.000035,...,0.000087,0.000052,24.344957,1.940736,2006.409,8.762778e-05,5.144445e-05,22.518183,1.635625,2006.409
4,758883089249339591,6182,648,1,758883089249337344,2247,331.801543,41.390926,0.000016,0.000015,...,0.000001,0.000011,20.683393,0.059923,2006.409,-5.135833e-05,-6.525000e-06,20.291475,0.163408,2006.409
5,758883089249337441,6182,648,1,758883089249337344,97,331.797318,41.390873,0.000070,0.000068,...,-0.000085,-0.000045,20.569685,0.132147,2006.409,5.000000e-07,-1.944444e-07,24.712810,0.622408,2006.409
6,758883089249337437,6182,648,1,758883089249337344,93,331.798537,41.392072,0.000033,0.000032,...,0.000005,-0.000006,20.174925,0.120331,2006.409,-4.353056e-05,3.612139e-04,24.027607,1.432995,2006.409
7,758883089249337419,6182,648,1,758883089249337344,75,331.792733,41.392017,0.000014,0.000013,...,-0.000004,-0.000005,15.883669,0.003775,2006.409,-4.591667e-06,-1.697222e-06,15.563893,0.005426,2006.409
8,758883089249337423,6182,648,1,758883089249337344,79,331.796938,41.394596,0.000015,0.000014,...,-0.000003,-0.000026,19.837260,0.032767,2006.409,-6.108333e-05,-6.421389e-05,19.305706,0.074711,2006.409
9,758883089249339570,6182,648,1,758883089249337344,2226,331.775376,41.392066,0.000018,0.000018,...,0.000031,0.000014,NaN,NaN,2006.409,-4.472222e-07,7.008333e-06,NaN,NaN,2006.409


Podemos obtener los detalles de un perfil en particular incluyendo el nombre del perfil como argumento en el método *list_profiles*:

In [26]:
qc.list_profiles('default')

{'description': 'default-db03',
 'type': 'hidden',
 'database': 'tapdb',
 'mydb_user': 'datalab',
 'accessURL': 'http://gp01.datalab.noirlab.edu:8080/ivoa-dal/tap',
 'vosRoot': 'vos://datalab.noirlab!vospace',
 'tempfilePath': '/net/dl2/temp',
 'resultStorePath': '/net/dl1/tap_data/resultStoreImpl'}

Entonces, intentemos realizar una consulta con el perfil predeterminado: obtengamos una lista de todas las tablas en la base de datos predeterminada.  Tenga en cuenta que en este caso estamos accediendo a la tabla *information_schema* de la base de datos, esta tabla no está incluida en el servicio TAP y por lo tanto **debemos** usar el argumento *sql* para hablar directamente con la base de datos.

In [ ]:
sql = 'SELECT table_catalog, table_schema, table_name FROM information_schema.tables'
try:
    qc.set_profile('default')
    default = qc.query(sql=sql)
except Exception as e:
    print (e.message)
else:
    print (default)